# 1. 讀取檔案

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

# 新增這兩個強大的套件
from xgboost import XGBClassifier
from sklearn.cluster import KMeans

# 讀取訓練與測試資料
df = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

# 觀察資料規模與前五筆資料
print(f"Train shape: {df.shape}, Test shape: {df_test.shape}")
df.head()

Train shape: (5634, 21), Test shape: (1409, 20)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4950-BDEUX,Male,0,No,No,35,No,No phone service,DSL,No,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,49.20,1701.65,No
1,7993-NQLJE,Male,0,Yes,Yes,15,Yes,No,Fiber optic,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,75.10,1151.55,No
2,7321-ZNSLA,Male,0,Yes,Yes,13,No,No phone service,DSL,Yes,...,No,Yes,No,No,Two year,No,Mailed check,40.55,590.35,No
3,4922-CVPDX,Female,0,Yes,No,26,Yes,No,DSL,No,...,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),73.50,1905.70,No
4,2903-YYTBW,Male,0,Yes,Yes,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,44.55,44.55,No


# 2.資料預處理

In [3]:
# --- 處理 TotalCharges 的髒資料 ---
# errors='coerce' 會將無法轉成數字的 " " 變為 NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df_test['TotalCharges'] = pd.to_numeric(df_test['TotalCharges'], errors='coerce')

# 填補缺失值 (用平均數填補)
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].mean())
df_test['TotalCharges'] = df_test['TotalCharges'].fillna(df_test['TotalCharges'].mean())

# 將目標欄位 Churn (Yes/No) 轉換為數值 (1/0)
target = 'Churn'
df[target] = df[target].apply(lambda x: 1 if x == 'Yes' else 0)

print("資料預處理完成：TotalCharges 已修復。")

資料預處理完成：TotalCharges 已修復。


# 3.特徵工程

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 1. 選擇要使用的數值特徵來做 K-Means
kmeans_features = ['tenure', 'MonthlyCharges', 'TotalCharges']

# 標準化 (K-Means 對數值範圍很敏感，所以要先標準化)
scaler = StandardScaler()
X_kmeans = scaler.fit_transform(df[kmeans_features])
X_test_kmeans = scaler.transform(df_test[kmeans_features])

# 2. 進行 K-Means 分群 (假設分成 5 群)
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
df['Cluster'] = kmeans.fit_predict(X_kmeans)
df_test['Cluster'] = kmeans.predict(X_test_kmeans)
print("特徵工程：K-Means 分群特徵 'Cluster' 已建立。")

# 3. 處理類別型欄位 (One-Hot Encoding)
# 排除不需要的欄位 (ID 和 Target)
drop_cols = ['customerID', 'Churn']
categorical_cols = [c for c in df.columns if df[c].dtype == 'object' and c not in drop_cols]

# 這裡我們使用 pandas 的 get_dummies 快速轉換
X_all = df.drop(columns=drop_cols)
X_test_all = df_test.drop(columns=['customerID']) # 測試集沒有 Churn

# 對訓練集做編碼
X = pd.get_dummies(X_all, columns=categorical_cols, drop_first=True)

# 對測試集做編碼 (重要：需確保測試集的欄位跟訓練集完全一樣)
X_test_final = pd.get_dummies(X_test_all, columns=categorical_cols, drop_first=True)
# 使用 reindex 補齊測試集可能缺少的欄位，缺少的補 0
X_test_final = X_test_final.reindex(columns=X.columns, fill_value=0)

y = df[target]
print(f"特徵處理完成。訓練集特徵數: {X.shape[1]}")

特徵工程：K-Means 分群特徵 'Cluster' 已建立。
特徵處理完成。訓練集特徵數: 31


# 4. 訓練模型

In [6]:
# 切分訓練集與驗證集 (Validation Set)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("開始訓練模型...")

# 計算正負樣本比例，用於設定 scale_pos_weight
ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

# 建立 XGBoost 模型
# scale_pos_weight={ratio}: 讓模型更關注 '流失(1)' 的樣本
# n_estimators=100: 樹的數量
# max_depth=4: 樹的深度，避免過擬合
model = XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    scale_pos_weight=ratio, 
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(X_train, y_train)

# 進行預測 (輸出機率值)
y_pred_proba = model.predict_proba(X_val)[:, 1]

# 計算並印出本地驗證分數
val_auc = roc_auc_score(y_val, y_pred_proba)
print(f"模型訓練完成！")
print(f"本地驗證分數 (Validation AUC): {val_auc:.4f}")

開始訓練模型...
模型訓練完成！
本地驗證分數 (Validation AUC): 0.8361


/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [14:45:21] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


# 5.輸出提交檔案

In [ ]:
# 使用訓練好的模型對測試集進行預測
predictions = model.predict_proba(X_test_final)[:, 1]

# 建立提交檔案
submission = pd.DataFrame({'customerID': df_test['customerID'], 'Churn': predictions})

# 存檔
submission.to_csv('submission.csv', index=False)
print("提交文件 'submission.csv' 已成功生成！")

提交文件 'submission.csv' 已成功生成！


# 6. 報告

姓名：林宥臣 學號：111303547

第一部分：準確度分數 (Accuracy Scores) (1分)  
我的準確度分數：0.83557 

第二部分：我的實驗記錄 (My Experiment Log) (3分)  
請記錄你做了哪些嘗試來提升分數，至少記錄兩次不同的嘗試。  
【實驗 1】  
    我做的修改：更換模型演算法並處理資料不平衡。將原本的 Decision Tree 替換為 Gradient Boosting (XGBoost/HistGradientBoosting)，利用集成學習提升預測力。同時設定權重參數（scale_pos_weight 或 class_weight='balanced'），讓模型加強對「流失（Churn=1）」樣本的學習權重。  
    結果與觀察 (分數變化、心得等)：分數突破 0.83。單一決策樹容易過擬合，而梯度提升樹（Gradient Boosting）能有效捕捉非線性關係。此外，處理不平衡問題後，模型不再只傾向預測「不流失」，大幅提升了 AUC 表現。  
    該次實驗分數： 0.83557  
【實驗 2】  
    我做的修改：未做實驗 2
    結果與觀察 (分數變化、心得等)：未做實驗 2
    該次實驗分數：未做實驗 2  

第三部分：總結與心得 (Conclusion & Reflection) (2分)  
請撰寫一段約 50-100 字的心得總結。內容需包含：  
(1) 你認為本次實驗中，提升準確率最有效的修改是什麼。  
(2) 這次不斷嘗試與修正的過程，帶給你最大的學習與啟發。  
內容：
    (1) 最有效的修改： 我認為本次實驗中，最有效的修改是 「更換為 Gradient Boosting 模型」配合「One-Hot Encoding」。因為電信資料包含大量類別特徵，若不編碼則無法利用；而梯度提升演算法在處理這類結構化表格資料時，準確度遠勝基礎決策樹。

    (2) 最大的學習與啟發： 這次過程讓我學到資料科學流程中「特徵工程」的重要性不亞於模型調整。一開始若未處理 TotalCharges 的髒資料，模型根本跑不動。這讓我體會到，一個好的預測模型，必須建立在乾淨且經過精心設計（如 K-Means 分群）的資料之上，才能發揮最大效能。